In [61]:
import datetime

from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, landscape
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
from reportlab.lib.units import cm
from reportlab.lib.utils import ImageReader
from reportlab.platypus import PageBreak
from reportlab.platypus import SimpleDocTemplate, Spacer, Table, TableStyle
from reportlab.rl_config import defaultPageSize
from reportlab.pdfgen import canvas

## Just try sample #1 from the doc to make sure everything works

In [62]:
c=canvas.Canvas("test.pdf", pagesize=letter)
c.drawString(100,100,"Hello World")
c.showPage()
c.save()

## Next lets draw something with the path object interface

In [63]:
c=canvas.Canvas("test.pdf", pagesize=letter)
width,height = letter
path=c.beginPath()
path.moveTo(width/2,height/2)
path.lineTo(width,height)
path.moveTo(width/2,height/2)
path.lineTo(0,height)
c.drawPath(path, stroke=1, fill=0)
c.showPage()
c.save()


## Lets draw a sparring tree for up to 8 people

In [64]:
def DrawBox(path,left,top):
    path.moveTo(left*cm,top*cm)
    path.lineTo((left-.3)*cm,(top*cm))
    path.lineTo((left-.3)*cm,(top-.3)*cm)
    path.lineTo(left*cm,(top-.3)*cm)
    path.lineTo(left*cm,top*cm)
                
    

In [65]:
def DrawBoxes(path,left,top):
    DrawBox(path,left,top) 
    DrawBox(path,left+.8, top)
    DrawBox(path,left+1.6, top)

### setup an array of bracket positions to represent how people map into a bracket

In [66]:
#braketpositions
# each bracket coordiate contains #people in ring, # of people in 1st round, second round starting position
# None is used as a placeholder if there is no people to put in the second round
# it doesn't make sense to have a 1 person bracket so it is not included
BRAKET_COORDINATES=[[0,0,None],  #placeholder to make array indexing work
                    [1,1,None],  #placeholder to make array indexing work
                    [2,2,None],
                    [3,2,1],
                    [4,4,None],
                    [5,2,1],
                    [6,4,2],
                    [7,6,3],
                    [8,8,None],
                    [9,2,1],
                    [10,4,2],
                    [11,6,3],
                    [12,8,4],
                    [13,10,5],
                    [14,12,6],
                    [15,14,7],
                    [16,15,None]]
                        

In [67]:
# test the continuity of the above table
def test_continuity():
    for i in range(len(BRAKET_COORDINATES)-1):
        assert BRAKET_COORDINATES[i][0] == BRAKET_COORDINATES[i+1][0]-1

def test_second_round_start_position_consistency():
    for i in range(len(BRAKET_COORDINATES)):
        if(BRAKET_COORDINATES[i][2]!=None):
            assert BRAKET_COORDINATES[i][2] == BRAKET_COORDINATES[i][1]/2
    
def test_braket_coordinates_integrity():
  test_continuity();
  test_second_round_start_position_consistency();
                   
test_braket_coordinates_integrity()

### write a function to return the column and row indexes for a ring given the number of compettitors

In [68]:
def CoordinatesForTextInBraket(competitors,index):
    compettitors_in_column1=BRAKET_COORDINATES[competitors][1]
    #print('competitors in column 1:',compettitors_in_column1)
    second_column_starting_index=BRAKET_COORDINATES[competitors][2]
    #print('starting index for column 2:',second_column_starting_index)
    if(index<compettitors_in_column1):
        r=index+1
        c=1
    else:
        r=second_column_starting_index + (index-compettitors_in_column1) +1
        c=2
    x=c
    y=r
    return x,y

In [69]:
## test the above funciton
people_in_ring=6
for i in range(people_in_ring):
    x,y=CoordinatesForTextInBraket(people_in_ring,i)
    print(x,y)

1 1
1 2
1 3
1 4
2 3
2 4


In [77]:
#setup contants for where the text goes
SPACE_BELOW_TEXT = 0.1
#textCoords=[[0.375,1.5+.0625],[0.375,2.375+.0625],[0.375,3.375+.0625],[0.375,4.25+.0625],[0.375,5.25+.0625],[0.375,6.125+.0625],[0.375,7.25+.0625],[0.375,8.125+.0625]]
'''
textCoords=[[1,20.6+SPACE_BELOW_TEXT],
            [1,18.3+SPACE_BELOW_TEXT],
            [1,15.8+SPACE_BELOW_TEXT],
            [1,13.5+SPACE_BELOW_TEXT],
            [1,10.9+SPACE_BELOW_TEXT],
            [1,8.6+SPACE_BELOW_TEXT],
            [1,6+SPACE_BELOW_TEXT],
            [1,3.8+SPACE_BELOW_TEXT]]
'''
initialTextCoords=[[.9,6+SPACE_BELOW_TEXT],
                   [.9,3.8+SPACE_BELOW_TEXT]]

textCoords=[]
for i in range(4):
    offset=i*4.8
    textCoords.append([initialTextCoords[1][0],initialTextCoords[1][1]+offset])
    textCoords.append([initialTextCoords[0][0],initialTextCoords[0][1]+offset])
textCoords.reverse()

secondColumnTextCoords=[]
initialSecondColumnTextCoords=[7.2+SPACE_BELOW_TEXT,19.4+SPACE_BELOW_TEXT]
for i in range(4):
    offset=i*4.8
    secondColumnTextCoords.append([initialSecondColumnTextCoords[0],initialSecondColumnTextCoords[1]-offset])
    



names=['Veronica Webster',
       'Rex Dunlop',
       'Jeff Shofner',
       'Larry Schwer',
       'Jewell Kinlaw',
       'Sydney Elliston']
#       'Shannon Alonso']
#       'Raymon Welborn']


### return the physical x,y coordinate for the text of a compettitors name based on the Bracket coordinates and the textcoordinates

In [78]:
def PhysicalCoordinates(number_of_compettitors,index):
        x,y=CoordinatesForTextInBraket(people_in_ring,i)
        print(x,y)
        if(x==1):
            px=textCoords[y-1][0]
            py=textCoords[y-1][1]
        else:
            px=secondColumnTextCoords[y-1][0]
            py=secondColumnTextCoords[y-1][1]
        return px,py

In [79]:
## test the above funciton
people_in_ring=6
for i in range(people_in_ring):
    px,py=PhysicalCoordinates(people_in_ring,i)
    print(px,py)

1 1
0.9 20.5
1 2
0.9 18.299999999999997
1 3
0.9 15.7
1 4
0.9 13.5
2 3
7.3 9.9
2 4
7.3 5.100000000000001


In [80]:
c=canvas.Canvas("8PersonTree.pdf", pagesize=letter)  #defaults to 72 pointer per inch

path=c.beginPath()

#determine where the orgin is
#path.moveTo(0,0) 
#path.lineTo(2*72,2*72) 

# First bracket
for i in range(4):
    offset=i*4.8
    path.moveTo(.8*cm,(3.8+offset)*cm) # 3/4 inch to the right, 1.5 inches up
    path.lineTo(6*cm,(3.8+offset)*cm) 
    path.lineTo(7.2*cm,(5+offset)*cm) 
    path.lineTo(6*cm,(6+offset)*cm)
    path.lineTo(.8*cm,(6+offset)*cm)
    DrawBoxes(path,1.5,5.3+offset) # 9/16 inches to the right and 2.125 inches up
    DrawBoxes(path,1.5,7.5+offset) # 9/16 inches to the right and 4 inches up
'''    
# first bracket
path.moveTo(.8*cm,3.8*cm) # 3/4 inch to the right, 1.5 inches up
path.lineTo(6*cm,3.8*cm) 
path.lineTo(7.2*cm,5*cm) 
path.lineTo(6*cm,6*cm)
path.lineTo(.8*cm,6*cm)

DrawBoxes(path,.6,5.3) # 9/16 inches to the right and 2.125 inches up
DrawBoxes(path,.6,7.5) # 9/16 inches to the right and 4 inches up

path.moveTo(.8*cm,8.6*cm) # 3/4 inch to the right, 3.375 inches up
path.lineTo(6*cm,8.6*cm) 
path.lineTo(7.2*cm,9.8*cm) 
path.lineTo(6*cm,10.9*cm)
path.lineTo(.8*cm,10.9*cm)

DrawBoxes(path,.6,10.1) # 9/16 inches to the right and 4 inches up
DrawBoxes(path,.6,12.4) # 9/16 inches to the right and 4 inches up


path.moveTo(.8*cm,13.5*cm) # 3/4 inch to the right, 5.25 inches up
path.lineTo(6*cm,13.5*cm) 
path.lineTo(7.2*cm,14.7*cm) 
path.lineTo(6*cm,15.8*cm)
path.lineTo(.8*cm,15.8*cm)

DrawBoxes(path,.6,15) # 9/16 inches to the right and 4 inches up
DrawBoxes(path,.6,17.3) # 9/16 inches to the right and 4 inches up

path.moveTo(.8*cm,18.3*cm) # 3/4 inch to the right, 7.25 inches up
path.lineTo(6*cm,18.3*cm) 
path.lineTo(7.2*cm,19.5*cm) 
path.lineTo(6*cm,20.6*cm)
path.lineTo(.8*cm,20.6*cm)

DrawBoxes(path,.6,19.9) # 9/16 inches to the right and 4 inches up
DrawBoxes(path,.6,21.8) # 9/16 inches to the right and 4 inches up
'''
# Second bracket
for i in range(2):
    offset=i*9.6
    path.moveTo(7.2*cm,(5+offset)*cm)  # 2.75 inch to the right, 2 inches up
    path.lineTo(12.4*cm,(5+offset)*cm) 
    path.lineTo(15.2*cm,(7.5+offset)*cm) 
    path.lineTo(12.4*cm,(9.8+offset)*cm)
    path.lineTo(7.2*cm,(9.8+offset)*cm)
    DrawBoxes(path,7.5,6.4+offset) # 9/16 inches to the right and 4 inches up
    DrawBoxes(path,7.5,11.3+offset) # 9/16 inches to the right and 4 inches up

'''
    
#second bracket
path.moveTo(7.2*cm,5*cm)  # 2.75 inch to the right, 2 inches up
path.lineTo(12.4*cm,5*cm) 
path.lineTo(15.2*cm,7.5*cm) 
path.lineTo(12.4*cm,9.8*cm)
path.lineTo(7.2*cm,9.8*cm)

DrawBoxes(path,7.5,6.4) # 9/16 inches to the right and 4 inches up
DrawBoxes(path,7.2,11.3) # 9/16 inches to the right and 4 inches up

path.moveTo(7.2*cm,14.7*cm)  # 5.75 inch to the right, 2 inches up
path.lineTo(12.4*cm,14.7*cm) 
path.lineTo(15.2*cm,17.2*cm) 
path.lineTo(12.4*cm,19.5*cm)
path.lineTo(7.2*cm,19.5*cm)

DrawBoxes(path,7.5,16.2) # 9/16 inches to the right and 4 inches up
DrawBoxes(path,7.5,21) # 9/16 inches to the right and 4 inches up
'''

#third bracket
path.moveTo(15.2*cm,7.5*cm)  # 2.9375 inch to the right, 6 inches up
path.lineTo(20.4*cm,7.5*cm) 
path.moveTo(15.2*cm,17.1*cm)
path.lineTo(20.4*cm,17.1*cm)

DrawBoxes(path,15.5,9) # 9/16 inches to the right and 4 inches up
DrawBoxes(path,15.5,18.7) # 9/16 inches to the right and 4 inches up

#draw the names
'''  first attempt - put everone in first column unless there is an odd man out
if(len(names) %2 == 0):
    for i, name in enumerate(names):   # using enumerate rather than zipping the lists together to handle less names than we have slots for
        c.drawString(textCoords[i][0]*cm,textCoords[i][1]*cm,name)
else:
    lastName=names[-1]
    names.remove(lastName)
    for i, name in enumerate(names):   # using enumerate rather than zipping the lists together to handle less names than we have slots for
        c.drawString(textCoords[i][0]*cm,textCoords[i][1]*cm,name)
    c.drawString(7.2*cm,(19.4+SPACE_BELOW_TEXT)*cm,lastName)   
'''
''' second attempt - simply put people in both columns
column1Index=0
column2Index=0
for i, name in enumerate(names):
    if(i<3):
      c.drawString(textCoords[column1Index][0]*cm,textCoords[column1Index][1]*cm,name)
      column1Index+=1  
    else:
      c.drawString(secondColumnTextCoords[column2Index][0]*cm,secondColumnTextCoords[column2Index][1]*cm,name)
      column2Index+=1  
'''
# third attempt use the PhysicalCoordinates(people_in_ring,i) function to get the coords to draw the names
people_in_ring=len(names)
for i, name in enumerate(names):
    px,py=PhysicalCoordinates(people_in_ring,i)
    c.drawString(px*cm,py*cm,name)


c.drawPath(path, stroke=1, fill=0)
c.showPage()
c.save()


1 1
1 2
1 3
1 4
2 3
2 4


## draw a sparring tree for up to 9 or more people

In [423]:
def DrawBoxCM(path,left,top):
    #print(left/cm,top/cm)
    top=top+1
    path.moveTo(left*cm,top*cm)
    path.lineTo((left-.2)*cm,(top*cm))
    path.lineTo((left-.2)*cm,(top-.2)*cm)
    path.lineTo(left*cm,(top-.2)*cm)
    path.lineTo(left*cm,top*cm)
                
    

In [424]:
def DrawBoxesCM(path,left,top):
    DrawBoxCM(path,left,top) 
    DrawBoxCM(path,left+.3, top)
    DrawBoxCM(path,left+.6, top)

In [425]:
#setup contants for where the text goes
SPACE_BELOW_TEXT = 0.1
initialTextCoords=[[0.8,1.5+SPACE_BELOW_TEXT],
            [0.8,2.65+SPACE_BELOW_TEXT]]

textCoords=[]
for i in range(8):
    offset=i*2.1
    textCoords.append([initialTextCoords[0][0],initialTextCoords[0][1]+offset])
    textCoords.append([initialTextCoords[1][0],initialTextCoords[1][1]+offset])
textCoords.reverse()

names=['Veronica Webster',
       'Rex Dunlop',
       'Jeff Shofner',
       'Larry Schwer',
       'Jewell Kinlaw',
       'Sydney Elliston',
       'Shannon Alonso',
       'Raymon Welborn',
       'Christoper Zaragoza',
       'Lekisha Champine',
       'Randa Widner',
       'Geoffrey Rucks',
       'Christa Harcrow',
       'Hayden Anstett',
       'Randal Mahoney']
#       'Mariano Ogletree']
#,'Eusebio Wolter','Moriah Bonaparte'

In [426]:
from reportlab.lib.units import cm

c=canvas.Canvas("16PersonTree.pdf", pagesize=landscape(letter))  #defaults to 72 pointer per inch
SPACE_BELOW_TEXT = .15 #centimeters

path=c.beginPath()

# first bracket
for i in range(8):
    offset=i*2.1
    path.moveTo(.8*cm,(1.5+offset)*cm) # .8 centimeters to the right, (1 + offset) centimeters up
    path.lineTo(6.2*cm,(1.5+offset)*cm) 
    path.lineTo(7.1*cm,(2+offset)*cm)
    path.lineTo(6.2*cm,(2.6+offset)*cm)
    path.lineTo(.8*cm,(2.6+offset)*cm)
    DrawBoxesCM(path,1.5,1.5+offset)
    DrawBoxesCM(path,1.5,(2.5+offset))

# second bracket
for i in range(4):
    offset=i*4.2
    path.moveTo(7.1*cm,(2+offset)*cm) # 7.2 centimeters to the right, (2 + offset) centimeters up
    path.lineTo(12.6*cm,(2+offset)*cm) 
    path.lineTo(13.3*cm,(3.1+offset)*cm)
    path.lineTo(12.6*cm,(4.1+offset)*cm)
    path.lineTo(7.1*cm,(4.1+offset)*cm)
    DrawBoxesCM(path,7.5,2+offset)
    DrawBoxesCM(path,7.5,4.1+offset)

# third bracket
for i in range(2):
    offset=i*8.4
    path.moveTo(13.3*cm,(3.1+offset)*cm) # 13.3 centimeters to the right, (3.1 + offset) centimeters up
    path.lineTo(18.7*cm,(3.1+offset)*cm) 
    path.lineTo(20.1*cm,(5.2+offset)*cm)
    path.lineTo(18.7*cm,(7.3+offset)*cm)
    path.lineTo(13.3*cm,(7.3+offset)*cm)
    DrawBoxesCM(path,14,3.1+offset)
    DrawBoxesCM(path,14,7.3+offset)
    
# fourth bracket
offset=0
path.moveTo(20.1*cm,5.2*cm) # 20.1 centimeters to the right, 5.2 centimeters up
path.lineTo(25.5*cm,5.2*cm) 
path.lineTo(27*cm,9.4*cm)
path.lineTo(25.5*cm,13.6*cm)
path.lineTo(20.1*cm,13.6*cm)
DrawBoxesCM(path,21,5.2)
DrawBoxesCM(path,21,13.6)

# winner line
path.moveTo(20.1*cm,9.4*cm) # 20.1 centimeters to the right, 9.4 centimeters up
path.lineTo(27*cm,9.4*cm)

if(len(names) %2 == 0):
    for i, name in enumerate(names):   # using enumerate rather than zipping the lists together to handle less names than we have slots for
        c.drawString(textCoords[i][0]*cm,textCoords[i][1]*cm,name)
else:
    lastName=names[-1]
    names.remove(lastName)
    for i, name in enumerate(names):   # using enumerate rather than zipping the lists together to handle less names than we have slots for
        c.drawString(textCoords[i][0]*cm,textCoords[i][1]*cm,name)
    c.drawString(7.1*cm,(16.8+SPACE_BELOW_TEXT)*cm,lastName)   

c.drawPath(path, stroke=1, fill=0)
c.showPage()
c.save()